# Text2SQL Agentic AI System - CrewAI Implementation

## Project Overview

This notebook implements a **Text2SQL Agentic AI System** using CrewAI framework that:
- Converts natural language banking questions into safe, read-only SQLite queries
- Enforces strict guardrails (read-only, no SELECT *, syntax validation)
- Returns concise, grounded natural language answers (no SQL code visible)
- Uses Azure OpenAI GPT-4.1-mini as the LLM (as per project requirements)

## Mandatory Deliverables

1. **code.ipynb** (this notebook) - Complete implementation with all required components
2. **submission.csv** - Responses to 10 test queries (query, response columns)

## Architecture

The system uses a **single ReAct agent** with 4 SQL tools:
1. `sql_db_list_tables` - Discover available tables
2. `sql_db_schema` - Inspect table schemas
3. `sql_db_query_checker` - Validate SQL queries (enforces guardrails)
4. `sql_db_query` - Execute validated queries

The agent follows a mandatory workflow sequence enforced through Task description.


## Section 1: Database Setup

In this section, we:
- Load the SQLite database using LangChain's SQLDatabase
- Verify connectivity
- Display table structure and sample data

In [ ]:
# Import required libraries
import os
import warnings
import pandas as pd
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Suppress ipywidgets warning (optional - install ipywidgets if you need Jupyter widget support)
warnings.filterwarnings('ignore', message='.*ipywidgets.*', category=UserWarning)

print("Libraries imported successfully!")

Libraries imported successfully!


In [55]:
# Load SQLite database using LangChain SQLDatabase
db_path = "banking_insights.db"

# Create SQLDatabase object
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

print(f"Database loaded successfully from: {db_path}")
print(f"Database dialect: {db.dialect}")

Database loaded successfully from: banking_insights.db
Database dialect: sqlite


In [56]:
# Verify connectivity and list all tables
tables = db.get_usable_table_names()
print(f"Found {len(tables)} tables in the database:")
for table in tables:
    print(f"  - {table}")

# Display sample rows from each table
print("\n" + "="*60)
print("Sample Data from Tables:")
print("="*60)

for table in tables:
    try:
        sample_query = f"SELECT * FROM {table} LIMIT 3"
        result = db.run(sample_query)
        print(f"\n{table} (first 3 rows):")
        print(result)
    except Exception as e:
        print(f"\n{table}: Error - {e}")

Found 6 tables in the database:
  - Account
  - Branch
  - Customer
  - Dispute
  - Merchant
  - Transactions

Sample Data from Tables:

Account (first 3 rows):
[(1001, 1, 2, 'Checking', '2022-07-03', 'Closed'), (1002, 2, 1, 'Checking', '2021-01-16', 'Active'), (1003, 2, 3, 'Savings', '2023-02-19', 'Active')]

Branch (first 3 rows):
[(1, 'Mumbai Fort', 'Mumbai', 'MH'), (2, 'Bengaluru MG Road', 'Bengaluru', 'KA'), (3, 'Delhi CP', 'New Delhi', 'DL')]

Customer (first 3 rows):
[(1, 'Arjun Mehta', 'Bengaluru', 'KA'), (2, 'Advait Das', 'New Delhi', 'DL'), (3, 'Kabir Gupta', 'New Delhi', 'DL')]

Dispute (first 3 rows):
[('D001', 'TDVMLQUCHO', '2025-06-03', 'Fraudulent', 'Resolved', 'Insufficient evidence'), ('D002', 'TV7J65MBVW', '2025-07-24', 'Fraudulent', 'Open', None), ('D003', 'T97XWMDLIW', '2025-08-23', 'Fraudulent', 'Resolved', 'Insufficient evidence')]

Merchant (first 3 rows):
[(1, 'Reliance Retail'), (2, 'Amazon India'), (3, 'Flipkart')]

Transactions (first 3 rows):
[('T950N0UP4D',

In [57]:
# Display database schema overview
schema_context_full = db.get_table_info_no_throw(tables)
print("Database Schema Overview:")
print("="*60)
print(schema_context_full[:2000])  # Print first 2000 chars
print("\n... (schema continues)")

Database Schema Overview:

CREATE TABLE "Account" (
	"AccountID" INTEGER, 
	"CustomerID" INTEGER NOT NULL, 
	"BranchID" INTEGER NOT NULL, 
	"AccountType" TEXT NOT NULL, 
	"OpenDate" DATE NOT NULL, 
	"Status" TEXT NOT NULL, 
	PRIMARY KEY ("AccountID"), 
	FOREIGN KEY("CustomerID") REFERENCES "Customer" ("CustomerID"), 
	FOREIGN KEY("BranchID") REFERENCES "Branch" ("BranchID")
)

/*
3 rows from Account table:
AccountID	CustomerID	BranchID	AccountType	OpenDate	Status
1001	1	2	Checking	2022-07-03	Closed
1002	2	1	Checking	2021-01-16	Active
1003	2	3	Savings	2023-02-19	Active
*/


CREATE TABLE "Branch" (
	"BranchID" INTEGER, 
	"Name" TEXT NOT NULL, 
	"City" TEXT, 
	"State" TEXT, 
	PRIMARY KEY ("BranchID")
)

/*
3 rows from Branch table:
BranchID	Name	City	State
1	Mumbai Fort	Mumbai	MH
2	Bengaluru MG Road	Bengaluru	KA
3	Delhi CP	New Delhi	DL
*/


CREATE TABLE "Customer" (
	"CustomerID" INTEGER, 
	"Name" TEXT NOT NULL, 
	"City" TEXT, 
	"State" TEXT, 
	PRIMARY KEY ("CustomerID")
)

/*
3 rows from

## Section 2: Tool Definitions & Implementations

In this section, we:
- Create LangChain SQLDatabaseToolkit with all 4 tools
- Enhance QuerySQLCheckerTool to enforce project-specific guardrails (read-only, no SELECT *)
- Prepare tools for CrewAI agent integration

In [58]:
# Enhanced Query Checker Tool with Guardrails
# This wraps LangChain's QuerySQLCheckerTool with custom security checks

@tool
def sql_db_query_checker(sql: str) -> str:
    """
    Validates SQL query before execution. Enforces guardrails:
    - Read-only (SELECT only, no DDL/DML)
    - No SELECT * (must specify columns)
    - Valid SQLite syntax (checked by LangChain)
    - Correct table/column names
    
    Args:
        sql: SQL query string to validate
        
    Returns:
        Validation result message (VALID or INVALID with reason)
    """
    sql_upper = sql.upper().strip()
    
    # Remove common code block markers if present
    sql_clean = sql_upper
    if sql_clean.startswith("```"):
        # Remove markdown code blocks
        lines = sql.split("\n")
        sql_lines = [line for line in lines if not line.strip().startswith("```")]
        sql_clean = " ".join(sql_lines).upper()
    
    # Guardrail 1: Check for forbidden keywords (read-only enforcement)
    forbidden_keywords = ['INSERT', 'UPDATE', 'DELETE', 'CREATE', 'DROP', 'ALTER', 'TRUNCATE', 
                         'REPLACE', 'MERGE', 'EXEC', 'EXECUTE']
    
    for keyword in forbidden_keywords:
        # Check if keyword is in query (as statement keyword, not in comments)
        if keyword in sql_clean:
            # More precise: check if it's a statement keyword
            patterns = [f'{keyword} ', f'{keyword}\n', f'{keyword}\t', f'{keyword};']
            if any(pattern in sql_clean for pattern in patterns):
                return f"INVALID: Query contains forbidden keyword '{keyword}'. Only SELECT queries are allowed (read-only mode)."
    
    # Guardrail 2: Check for SELECT *
    if 'SELECT *' in sql_clean or 'SELECT*' in sql_clean:
        return "INVALID: SELECT * is not allowed. Specify column names explicitly (e.g., SELECT column1, column2 FROM table)."
    
    # Guardrail 3: Must start with SELECT
    if not sql_clean.startswith('SELECT'):
        return "INVALID: Query must be a SELECT statement (read-only)."
    
    # If all guardrails pass, use LangChain's checker for syntax validation
    try:
        # Check if LangChain checker is available (will be initialized in Section 4)
        # Access from globals since it's defined in a later cell
        if 'query_checker_langchain' in globals() and globals()['query_checker_langchain'] is not None:
            langchain_checker = globals()['query_checker_langchain']
            result = langchain_checker.invoke({"query": sql})
            # If LangChain checker returns SQL (corrected or original), validation passed
            if result and "INVALID" not in result.upper() and "ERROR" not in result.upper():
                return "VALID: Query passes all security checks and SQL syntax validation."
            else:
                return f"INVALID: {result}"
        else:
            # If LangChain checker not yet initialized, return basic validation
            return "VALID: Query passes security checks. (Syntax validation pending - LangChain checker will be available after Section 4)"
    except Exception as e:
        return f"INVALID: Syntax validation error - {str(e)}"

print("Enhanced Query Checker Tool created with guardrails!")

Enhanced Query Checker Tool created with guardrails!


## Section 3: System Instruction Prompt

In this section, we:
- Load schema context and example queries
- Create comprehensive Task description that includes:
  - Mandatory workflow sequence
  - Guardrails enforcement
  - Output format requirements
  - Database schema context
  - Few-shot examples

In [59]:
# Load sample queries with responses for few-shot examples
sample_queries_df = pd.read_csv("sample_queries_with_responses.csv")
print("Sample queries loaded:")
print(sample_queries_df)

Sample queries loaded:
                                         query  \
0  Top 5 merchants by debit spend in July 2025   
1        Top 5 customers by total debit spend    

                                            response  
0  The top 5 merchants by debit spend in July 202...  
1  The top 5 customers by total debit spend are:\...  


In [60]:
# Build schema context string (already loaded in Section 1)
# Build example queries context
example_queries_text = "\n\n".join([
    f"Example {i+1}:\nQuery: {row['query']}\nResponse: {row['response']}"
    for i, (idx, row) in enumerate(sample_queries_df.iterrows())
])

print("Schema and examples prepared for Task description.")

Schema and examples prepared for Task description.


## Section 4: LLM Integration

In this section, we:
- Set up Azure OpenAI API configuration (as per project requirements)
- Configure AzureChatOpenAI (GPT-4.1-mini) for LangChain toolkit
- Configure LLM for CrewAI agent

**Note:** This project requires Azure OpenAI API. Please ensure you have the following environment variables set:
- `AZURE_OPENAI_API_KEY`: Your Azure OpenAI API key
- `AZURE_OPENAI_ENDPOINT`: Your Azure OpenAI endpoint URL
- `MODEL_DEPLOYMENT_NAME`: Deployment name for GPT-4.1-mini (default: "gpt-4.1-mini")
- `OPENAI_API_VERSION`: API version (default: "2025-04-01-preview")
- `PROJECT_ID`: Project identifier (default: "TEST")

In [ ]:
# Azure OpenAI configuration (required for this project)
# Based on "Fractal Capstone Project - Environment Setup Guide"
azure_openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.environ.get("AZURE_OPENAI_API_KEY")  # Note: Guide uses AZURE_OPENAI_API_KEY

# Check if Azure OpenAI configuration is available
if not azure_openai_endpoint or not azure_openai_api_key:
    raise ValueError(
        "Azure OpenAI configuration is required for this project!\n"
        "Please set the following environment variables:\n"
        "  - AZURE_OPENAI_ENDPOINT: Your Azure OpenAI endpoint URL\n"
        "  - AZURE_OPENAI_API_KEY: Your Azure OpenAI API key\n"
        "  - MODEL_DEPLOYMENT_NAME: Deployment name (default: 'gpt-4.1-mini')\n"
        "  - OPENAI_API_VERSION: API version (default: '2025-04-01-preview')\n"
        "  - PROJECT_ID: Project identifier (default: 'TEST')\n"
    )

print("Azure OpenAI configuration detected!")
print(f"  Endpoint: {azure_openai_endpoint}")
print("  Using Azure OpenAI for LLM (as per project requirements)")



OpenAI API key configured successfully!
LangSmith tracing disabled to avoid timeout prompts.


In [ ]:
# Create LLM for LangChain toolkit and CrewAI agent
# Get Azure OpenAI configuration values (with defaults as per guide)
azure_api_version = os.environ.get("OPENAI_API_VERSION", "2025-04-01-preview")
model_deployment_name = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4.1-mini")
project_id = os.environ.get("PROJECT_ID", "TEST")

# Create Azure OpenAI LLM client
llm = AzureChatOpenAI(
    # your Azure OpenAI endpoint URL
    azure_endpoint=azure_openai_endpoint,
    # your Azure OpenAI LLM API version
    api_version=azure_api_version,
    # your GPT-4.1-mini model deployment name
    azure_deployment=model_deployment_name,
    temperature=0,
    # comment out the api_key if using AD tokens
    api_key=azure_openai_api_key,
    # uncomment the azure_ad_token line below if using AD token auth mechanism
    # azure_ad_token=token,
    default_headers={
        "projectId": project_id
    }
)

print("Azure OpenAI LLM configured successfully:")
print(f"  Deployment: {model_deployment_name}")
print(f"  API Version: {azure_api_version}")
print(f"  Project ID: {project_id}")
print(f"  Endpoint: {azure_openai_endpoint}")

LLM configured: gpt-4o-mini


In [63]:
# Now create LangChain SQLDatabaseToolkit with the LLM
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
langchain_tools = toolkit.get_tools()

print(f"LangChain Toolkit created with {len(langchain_tools)} tools:")
for langchain_tool in langchain_tools:
    print(f"  - {langchain_tool.name}")

# Extract individual LangChain tools by exact name matching
list_tables_langchain = None
schema_langchain = None
query_checker_langchain = None
query_langchain = None

for langchain_tool in langchain_tools:
    tool_name = langchain_tool.name.lower()
    if tool_name == 'sql_db_list_tables':
        list_tables_langchain = langchain_tool
    elif tool_name == 'sql_db_schema':
        schema_langchain = langchain_tool
    elif tool_name == 'sql_db_query_checker':
        query_checker_langchain = langchain_tool
    elif tool_name == 'sql_db_query':
        query_langchain = langchain_tool

# Verify all tools were found
print(f"\nExtracted LangChain tools:")
print(f"  - List tables: {list_tables_langchain is not None}")
print(f"  - Schema: {schema_langchain is not None}")
print(f"  - Query checker: {query_checker_langchain is not None}")
print(f"  - Query: {query_langchain is not None}")

# Global variable to capture SQL queries for extraction
_last_executed_sql_query = None

# Wrap LangChain tools as CrewAI tools using @tool decorator
# Following CrewAI documentation pattern for LangChain tool integration
# Note: We use @tool without arguments - tool name comes from function name
@tool
def sql_db_list_tables() -> str:
    """
    Lists all tables in the database.
    No input required.
    Returns a list of table names.
    """
    if list_tables_langchain:
        try:
            return list_tables_langchain.invoke({})
        except Exception as e:
            return f"Error listing tables: {str(e)}"
    return "Error: List tables tool not initialized"

@tool
def sql_db_schema(table_names: str) -> str:
    """
    Gets schema information for specified tables.
    
    Args:
        table_names: Comma-separated list of table names (e.g., "Customer,Account,Transactions")
    
    Returns:
        Schema information including column names, types, and sample rows
    """
    if schema_langchain:
        try:
            return schema_langchain.invoke({"table_names": table_names})
        except Exception as e:
            return f"Error getting schema: {str(e)}"
    return "Error: Schema tool not initialized"

@tool
def sql_db_query(query: str) -> str:
    """
    Executes a SQL query on the database and returns results.
    
    Args:
        query: SQL SELECT query to execute (must be validated first)
    
    Returns:
        Query results as formatted string
    """
    global _last_executed_sql_query
    # Store the SQL query for later extraction
    _last_executed_sql_query = query
    
    if query_langchain:
        try:
            return query_langchain.invoke({"query": query})
        except Exception as e:
            return f"Error executing query: {str(e)}"
    return "Error: Query tool not initialized"

# Create final tool list: all CrewAI-compatible tools
# Note: sql_db_query_checker is already defined in Cell 7 using @tool decorator
sql_tools = [sql_db_list_tables, sql_db_schema, sql_db_query_checker, sql_db_query]

print("\nFinal tool list prepared for CrewAI agent (all CrewAI-compatible):")
for i, tool in enumerate(sql_tools):
    if tool:
        tool_name = getattr(tool, 'name', f'Tool_{i}')
        print(f"  {i+1}. {tool_name}")
    else:
        print(f"  {i+1}. None (ERROR - tool not initialized)")

LangChain Toolkit created with 4 tools:
  - sql_db_query
  - sql_db_schema
  - sql_db_list_tables
  - sql_db_query_checker

Extracted LangChain tools:
  - List tables: True
  - Schema: True
  - Query checker: True
  - Query: True

Final tool list prepared for CrewAI agent (all CrewAI-compatible):
  1. sql_db_list_tables
  2. sql_db_schema
  3. sql_db_query_checker
  4. sql_db_query


## Section 5: Agent Creation

In this section, we:
- Create CrewAI Agent with tools and system configuration
- Create Task with comprehensive description (workflow, guardrails, examples)
- Create Crew with sequential process
- Define execution function

In [ ]:
# Create Text2SQL Agent
# Use the configured Azure OpenAI LLM from Cell 13
text2sql_agent = Agent(
    role="Text2SQL Banking Analyst",
    goal="Convert natural language banking questions into safe, read-only SQLite queries and return concise, grounded answers",
    backstory=(
        "You are an expert banking analyst with deep SQL expertise. "
        "You understand database schemas, can write efficient queries, "
        "and always follow strict security guardrails. "
        "You are methodical and always follow the required workflow sequence. "
        "When queries fail or return unexpected results, you analyze the error, re-examine the schema, "
        "and iteratively correct the query until it succeeds. You never give up after a single failed attempt. "
        "You are persistent and use your ReAct reasoning to identify and fix issues systematically. "
        "You never generate INSERT, UPDATE, DELETE, or DDL statements. "
        "You always specify column names explicitly (never use SELECT *). "
        "You provide natural language answers without showing SQL code to users."
    ),
    tools=sql_tools,
    llm=llm,  # Use the configured Azure OpenAI LLM from Cell 13
    verbose=True,
    max_iter=10,  # Allow enough iterations for full workflow
    allow_delegation=False,  # Single agent, no delegation needed
    max_execution_time=300  # 5 minutes max execution time
)

print("Text2SQL Agent created successfully!")

Text2SQL Agent created successfully!


In [65]:
# Create Task with comprehensive description including workflow, guardrails, and examples
text2sql_task = Task(
    description=(
        "You are a Text2SQL Banking Analyst. Answer the user's banking question: {query}\n\n"
        
        "=== MANDATORY WORKFLOW (MUST FOLLOW THIS EXACT SEQUENCE - ReAct Pattern) ===\n"
        "STEP 1: Use the list tables tool (sql_db_list_tables) to discover all available tables in the database.\n"
        "STEP 2: Use the schema tool (sql_db_schema) with relevant table names to inspect table schemas, column names, and data types.\n"
        "STEP 3: Based on the user's question and the schema information, draft a SQL query that answers the question.\n"
        "STEP 4: Use the query checker tool (sql_db_query_checker) to validate the SQL query. The query MUST pass validation before proceeding.\n"
        "STEP 5: If validation passes, use the query tool (sql_db_query) to execute the SQL query and get results from the database.\n"
        "  - CRITICAL RETRY LOGIC: If the query execution fails, returns an error, or returns unexpected/empty results when data should exist:\n"
        "    → Analyze the error message or unexpected result carefully\n"
        "    → Re-examine the schema (especially join paths and foreign key relationships)\n"
        "    → Go back to STEP 3: Draft a corrected SQL query based on your analysis\n"
        "    → Repeat STEPS 3→4→5 until the query executes successfully and returns expected results\n"
        "    → Only proceed to STEP 6 after a successful query execution with valid results\n"
        "    → Never give up after a single failed attempt - use iterative correction\n"
        "STEP 6: Analyze the query results and summarize them as a clear, natural language answer. Do NOT include SQL code in your final response.\n\n"
        
        "=== GUARDRAILS (STRICTLY ENFORCE) ===\n"
        "- Read-only: Only SELECT queries allowed (no INSERT, UPDATE, DELETE, CREATE, DROP, ALTER, or any DDL/DML)\n"
        "- No SELECT *: Never use SELECT * - always specify column names explicitly (e.g., SELECT column1, column2 FROM table)\n"
        "- Explicit JOINs: Use explicit JOIN ... ON ... syntax with correct foreign key relationships\n"
        "  CRITICAL JOIN PATHS (verify against schema before joining):\n"
        "  • Branch → Account → Transactions: Branch.BranchID = Account.BranchID, then Account.AccountID = Transactions.AccountID\n"
        "  • Customer → Account → Transactions: Customer.CustomerID = Account.CustomerID, then Account.AccountID = Transactions.AccountID\n"
        "  • NEVER join Branch directly to Transactions (Branch.BranchID ≠ Transactions.AccountID)\n"
        "  • NEVER join Customer directly to Transactions (Customer.CustomerID ≠ Transactions.AccountID)\n"
        "- NULL Handling: Transactions.MerchantID is nullable - use LEFT JOIN when you need all transactions, INNER JOIN when you only need transactions with merchants\n"
        "- Date/Time Format: Use SQLite date functions - strftime('%Y', TxnDate) for year, strftime('%Y-%m', TxnDate) for month, use BETWEEN for date ranges\n"
        "- Case Sensitivity: Match exact case for TEXT values - Status = 'Active' (not 'active'), TxnType = 'Credit' (not 'credit'), Dispute.Status = 'Open' (not 'open')\n"
        "- Column Disambiguation: Always use table aliases (e.g., B.Name AS BranchName, C.Name AS CustomerName) to avoid ambiguity when multiple tables have similar column names\n"
        "- Aggregation Safety: Use NULLIF(denominator, 0) when dividing to avoid division by zero errors\n"
        "- Result limits: Limit results to 10 rows unless the user specifically requests more\n"
        "- Valid syntax: Ensure SQL syntax is correct for SQLite database\n\n"
        
        "=== OUTPUT FORMAT ===\n"
        "- Provide a natural language answer (paragraph or bullet points) - NO SQL code in the response\n"
        "- For simple ranked lists (e.g., top 5, top 3), use numbered bullet points: '1. Item with value X', '2. Item with value Y'\n"
        "- Use tables ONLY for complex multi-column results (3+ columns) or when comparing multiple attributes side-by-side\n"
        "- For simple 2-column results (e.g., name + amount), prefer bullet points over tables\n"
        "- Brief explanation of findings\n"
        "- Format numbers using Indian numbering system: Place commas every 2 digits after the first 3 digits from the right. Examples: 1,002,190 → 10,02,190.00 INR, 965,256 → 9,65,256.00 INR. For very large numbers, use lakhs/crores (e.g., 1,00,00,000 → 1 Crore INR, 1,00,000 → 1 Lakh INR)\n\n"
        
        "=== DATABASE SCHEMA ===\n"
        f"{schema_context_full}\n\n"
        
        "=== EXAMPLE QUERIES ===\n"
        f"{example_queries_text}\n"
        "Note: The example responses use numbered bullet points (1., 2., 3.) for simple ranked lists. Follow this format for similar queries.\n\n"
        
        "Remember: Follow the workflow sequence strictly, enforce all guardrails, and provide natural language answers without SQL code."
    ),
    agent=text2sql_agent,
    expected_output=(
        "A clear, natural language answer to the banking question with optional table showing results. "
        "NO SQL code should be included in the response. The answer should be grounded in actual database results."
    )
)

print("Task created with comprehensive workflow and guardrails!")

Task created with comprehensive workflow and guardrails!


In [66]:
# Create Crew with sequential process
text2sql_crew = Crew(
    agents=[text2sql_agent],
    tasks=[text2sql_task],
    process=Process.sequential,  # Single task, sequential execution
    verbose=True
)

print("Crew created successfully!")

Crew created successfully!


In [67]:
# Define execution function with SQL query extraction
def run_text2sql_query(user_query: str, extract_sql=False):
    """
    Execute Text2SQL agent on user query.
    
    Args:
        user_query: Natural language banking question
        extract_sql: If True, also extract the SQL query that was executed
        
    Returns:
        If extract_sql=False: Natural language answer (string)
        If extract_sql=True: Tuple of (answer, sql_query)
    """
    inputs = {"query": user_query}
    result = text2sql_crew.kickoff(inputs=inputs)
    
    # Extract the final response from task output
    answer = result.tasks_output[0].raw
    
    sql_query = None
    if extract_sql:
        # Extract SQL query - use global variable captured by sql_db_query tool
        global _last_executed_sql_query
        if _last_executed_sql_query:
            sql_query = _last_executed_sql_query
            # Reset for next query
            _last_executed_sql_query = None
        else:
            # Fallback: Try to parse from result string representation
            try:
                import re
                result_str = str(result)
                # Look for SQL query patterns
                sql_patterns = [
                    r'"query"\s*:\s*"([^"]+)"',  # JSON format
                    r'(SELECT\s+.*?;)',  # Direct SQL pattern
                ]
                for pattern in sql_patterns:
                    matches = re.findall(pattern, result_str, re.DOTALL | re.IGNORECASE)
                    if matches:
                        potential_query = matches[-1].strip()
                        potential_query = potential_query.replace('\\n', '\n').replace('\\"', '"')
                        if potential_query.upper().startswith('SELECT') and len(potential_query) > 20:
                            sql_query = potential_query
                            break
            except:
                pass
    
    if extract_sql:
        return answer, sql_query
    return answer

print("Execution function defined with SQL extraction capability!")

Execution function defined with SQL extraction capability!


## Section 6: Agent Experimentation & Validation

In this section, we:
- Test the agent on 2 sample queries from `sample_queries_with_responses.csv`
- Compare generated responses with expected responses
- Verify workflow sequence is followed (inspect verbose logs)
- Verify guardrails are enforced
- Refine Task description if needed based on findings

In [68]:
# Test agent on sample queries
print("Testing agent on sample queries...\n")
print("="*80)

validation_results = []

for idx, row in sample_queries_df.iterrows():
    query = row['query']
    expected = row['response']
    
    print(f"\n{'='*80}")
    print(f"Test {idx + 1}")
    print(f"{'='*80}")
    print(f"Query: {query}\n")
    
    try:
        # Run the agent
        generated_response = run_text2sql_query(query)
        
        print(f"Generated Response:\n{generated_response}\n")
        print(f"Expected Response:\n{expected}\n")
        
        validation_results.append({
            'query': query,
            'generated': generated_response,
            'expected': expected
        })
        
    except Exception as e:
        print(f"Error processing query: {e}\n")
        validation_results.append({
            'query': query,
            'generated': f"ERROR: {str(e)}",
            'expected': expected
        })

print("\n" + "="*80)
print("Validation Complete!")
print("="*80)

Testing agent on sample queries...


Test 1
Query: Top 5 merchants by debit spend in July 2025



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 28ca515b-71a8-410a-8f09-464da8d5ab8d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Text2SQL Banking Analyst                                                                                │
│                                                                                                                 │
│  Task: You are a Text2SQL Banking Analyst. Answer the user's banking question: Top 5 merchants by debit spend   │
│  in July 2025                                                                                                   │
│                                                                                                                 │
│  === MANDATORY WORKFLOW (MUST FOLLOW THIS EXACT SEQUENCE - ReAct Pattern) ===                                   │
│  STEP 1: Use the list tables tool (sql_db_list_tables) to discover all available tables in the database.        │
│  STEP 2: Use the schema tool (sql_db_schema) with relevant table names to inspect table schemas, column names,  │
│  and data types.                                                                                                │
│  STEP 3: Based on the user's question and the schema information, draft a SQL query that answers the question.  │
│  STEP 4: Use the query checker tool (sql_db_query_checker) to validate the SQL query. The query MUST pass       │
│  validation before proceeding.                                                                                  │
│  STEP 5: If validation passes, use the query tool (sql_db_query) to execute the SQL query and get results from  │
│  the database.                                                                                                  │
│    - CRITICAL RETRY LOGIC: If the query execution fails, returns an error, or returns unexpected/empty results  │
│  when data should exist:                                                                                        │
│      → Analyze the error message or unexpected result carefully                                                 │
│      → Re-examine the schema (especially join paths and foreign key relationships)                              │
│      → Go back to STEP 3: Draft a corrected SQL query based on your analysis                                    │
│      → Repeat STEPS 3→4→5 until the query executes successfully and returns expected results                    │
│      → Only proceed to STEP 6 after a successful query execution with valid results                             │
│      → Never give up after a single failed attempt - use iterative correction                                   │
│  STEP 6: Analyze the query results and summarize them as a clear, natural language answer. Do NOT include SQL   │
│  code in your final response.                                                                                   │
│                                                                                                                 │
│  === GUARDRAILS (STRICTLY ENFORCE) ===                                                                          │
│  - Read-only: Only SELECT queries allowed (no INSERT, UPDATE, DELETE, CREATE, DROP, ALTER, or any DDL/DML)      │
│  - No SELECT *: Never use SELECT * - always specify column names explicitly (e.g., SELECT column1, column2      │
│  FROM table)                                                                                                    │
│  - Explicit JOINs: Use explicit JOIN ... ON ... syntax with correct foreign key relationships                   │
│    CRITICAL JOIN PATHS (verify against schema before joining):                                                  │
│    • Branch → Account → Transactions: Branch.BranchID = Account.BranchID, then Account.AccountID =              │
│  Transactions.AccountID                                

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Text2SQL Banking Analyst                                                                                │
│                                                                                                                 │
│  Thought: Action: sql_db_list_tables                                                                            │
│                                                                                                                 │
│  Using Tool: sql_db_list_tables                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {}                                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  [('Advait Das', 7276084.0), ('Kabir Gupta', 7082768.0), ('Kabir Das', 6187994.0), ('Rahul Khan', 5579585.0),   │
│  ('Dev Verma', 5161857.0)]                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Text2SQL Banking Analyst                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The top 5 customers by total debit spend are:                                                                  │
│                                                                                                                 │
│  1. Advait Das with a total debit spend of 72,76,084.00 INR                                                     │
│  2. Kabir Gupta with a total debit spend of 70,82,768.00 INR                                                    │
│  3. Kabir Das with a total debit spend of 61,87,994.00 INR                                                      │
│  4. Rahul Khan with a total debit spend of 55,79,585.00 INR                                                     │
│  5. Dev Verma with a total debit spend of 51,61,857.00 INR                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 47039ff1-1771-4e9f-b10b-80615c3a2b75                                                                     │
│  Agent: Text2SQL Banking Analyst                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 28ca515b-71a8-410a-8f09-464da8d5ab8d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The top 5 customers by total debit spend are:                                                    │
│                                                                                                                 │
│  1. Advait Das with a total debit spend of 72,76,084.00 INR                                                     │
│  2. Kabir Gupta with a total debit spend of 70,82,768.00 INR                                                    │
│  3. Kabir Das with a total debit spend of 61,87,994.00 INR                                                      │
│  4. Rahul Khan with a total debit spend of 55,79,585.00 INR                                                     │
│  5. Dev Verma with a total debit spend of 51,61,857.00 INR                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Generated Response:
The top 5 customers by total debit spend are:

1. Advait Das with a total debit spend of 72,76,084.00 INR
2. Kabir Gupta with a total debit spend of 70,82,768.00 INR
3. Kabir Das with a total debit spend of 61,87,994.00 INR
4. Rahul Khan with a total debit spend of 55,79,585.00 INR
5. Dev Verma with a total debit spend of 51,61,857.00 INR

Expected Response:
The top 5 customers by total debit spend are:
1. Advait Das with a total debit spend of 7,276,084.00 INR
2. Kabir Gupta with a total debit spend of 7,082,768.00 INR
3. Kabir Das with a total debit spend of 6,187,994.00 INR
4. Rahul Khan with a total debit spend of 5,579,585.00 INR
5. Dev Verma with a total debit spend of 5,161,857.00 INR


Validation Complete!


## Section 7: Testing on Hidden Queries & Submission Generation

In this section, we:
- Load 10 test queries from `test_queries.csv`
- Run the agent on each test query
- Generate `submission.csv` with query/response pairs
- Verify submission format (2 columns: query, response)

In [69]:
# Load test queries
test_queries_df = pd.read_csv("test_queries.csv")
print(f"Loaded {len(test_queries_df)} test queries")
print("\nTest queries:")
print(test_queries_df)

Loaded 11 test queries

Test queries:
                                                query  response
0        Top 5 merchants by debit spend in July 2025.       NaN
1              Count active accounts by account type.       NaN
2   List the top 3 spending categories by total tr...       NaN
3   Top 3 customers who spent the most on dining i...       NaN
4   Current balance for the top 5 accounts (all ti...       NaN
5               Give me the total deposits by branch.       NaN
6   Which branch has the highest number of customers.       NaN
7   Show the top 5 highest-value transactions in 2...       NaN
8   Top 5 merchants by number of disputes (all time).       NaN
9   Show transaction volume numbers and percentage...       NaN
10  Top 5 Average customer debit spend by merchant...       NaN


In [70]:
# Run agent on all test queries and collect responses with SQL queries
print("Running agent on test queries...\n")
print("="*80)

submission_results = []
query_sql_results = []
query_sql_insight_results = []

for idx, row in test_queries_df.iterrows():
    query = row['query']
    print(f"\nProcessing Query {idx + 1}/{len(test_queries_df)}: {query}")
    
    try:
        # Extract both answer and SQL query
        response, sql_query = run_text2sql_query(query, extract_sql=True)
        
        # Clean response - ensure no SQL code blocks remain
        response_clean = response
        if "```sql" in response_clean or "```" in response_clean:
            # Remove code blocks if any
            import re
            response_clean = re.sub(r'```[a-z]*\n.*?\n```', '', response_clean, flags=re.DOTALL)
            response_clean = response_clean.strip()
        
        # Clean SQL query if extracted
        if sql_query:
            sql_query = sql_query.strip()
            # Remove markdown code blocks if present
            if sql_query.startswith("```"):
                sql_query = re.sub(r'```[a-z]*\n', '', sql_query)
                sql_query = re.sub(r'\n```', '', sql_query)
                sql_query = sql_query.strip()
        else:
            sql_query = "SQL query not captured"
        
        # Store results for different CSV files
        submission_results.append({
            'query': query,
            'response': response_clean
        })
        
        query_sql_results.append({
            'query': query,
            'sql_query': sql_query
        })
        
        query_sql_insight_results.append({
            'query': query,
            'sql_query': sql_query,
            'insight': response_clean
        })
        
        print(f"✓ Completed (SQL: {'Captured' if sql_query != 'SQL query not captured' else 'Not captured'})")
        
    except Exception as e:
        print(f"✗ Error: {str(e)}")
        error_msg = f"Error processing query: {str(e)}"
        submission_results.append({
            'query': query,
            'response': error_msg
        })
        query_sql_results.append({
            'query': query,
            'sql_query': "Error"
        })
        query_sql_insight_results.append({
            'query': query,
            'sql_query': "Error",
            'insight': error_msg
        })

print("\n" + "="*80)
print("All test queries processed!")
print("="*80)

Running agent on test queries...


Processing Query 1/11: Top 5 merchants by debit spend in July 2025.


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 28ca515b-71a8-410a-8f09-464da8d5ab8d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Text2SQL Banking Analyst                                                                                │
│                                                                                                                 │
│  Task: You are a Text2SQL Banking Analyst. Answer the user's banking question: Top 5 merchants by debit spend   │
│  in July 2025.                                                                                                  │
│                                                                                                                 │
│  === MANDATORY WORKFLOW (MUST FOLLOW THIS EXACT SEQUENCE - ReAct Pattern) ===                                   │
│  STEP 1: Use the list tables tool (sql_db_list_tables) to discover all available tables in the database.        │
│  STEP 2: Use the schema tool (sql_db_schema) with relevant table names to inspect table schemas, column names,  │
│  and data types.                                                                                                │
│  STEP 3: Based on the user's question and the schema information, draft a SQL query that answers the question.  │
│  STEP 4: Use the query checker tool (sql_db_query_checker) to validate the SQL query. The query MUST pass       │
│  validation before proceeding.                                                                                  │
│  STEP 5: If validation passes, use the query tool (sql_db_query) to execute the SQL query and get results from  │
│  the database.                                                                                                  │
│    - CRITICAL RETRY LOGIC: If the query execution fails, returns an error, or returns unexpected/empty results  │
│  when data should exist:                                                                                        │
│      → Analyze the error message or unexpected result carefully                                                 │
│      → Re-examine the schema (especially join paths and foreign key relationships)                              │
│      → Go back to STEP 3: Draft a corrected SQL query based on your analysis                                    │
│      → Repeat STEPS 3→4→5 until the query executes successfully and returns expected results                    │
│      → Only proceed to STEP 6 after a successful query execution with valid results                             │
│      → Never give up after a single failed attempt - use iterative correction                                   │
│  STEP 6: Analyze the query results and summarize them as a clear, natural language answer. Do NOT include SQL   │
│  code in your final response.                                                                                   │
│                                                                                                                 │
│  === GUARDRAILS (STRICTLY ENFORCE) ===                                                                          │
│  - Read-only: Only SELECT queries allowed (no INSERT, UPDATE, DELETE, CREATE, DROP, ALTER, or any DDL/DML)      │
│  - No SELECT *: Never use SELECT * - always specify column names explicitly (e.g., SELECT column1, column2      │
│  FROM table)                                                                                                    │
│  - Explicit JOINs: Use explicit JOIN ... ON ... syntax with correct foreign key relationships                   │
│    CRITICAL JOIN PATHS (verify against schema before joining):                                                  │
│    • Branch → Account → Transactions: Branch.BranchID = Account.BranchID, then Account.AccountID =              │
│  Transactions.AccountID                                

Repaired JSON: {}

[EventBus Error] Handler 'on_agent_logs_execution' failed for event 'AgentLogsExecutionEvent': Extra data: line 2 
column 1 (char 3)

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  [('Reliance Retail', 117148.31818181818), ('Amazon India', 114694.57777777778), ('Apple Store',                │
│  111275.31914893616), ('Uber', 107809.50847457627), ('IRCTC', 105685.6530612245)]                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Text2SQL Banking Analyst                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The top 5 average customer debit spend by merchant in 2025 are:                                                │
│                                                                                                                 │
│  1. Reliance Retail with an average spend of 1,17,148.32 INR                                                    │
│  2. Amazon India with an average spend of 1,14,694.58 INR                                                       │
│  3. Apple Store with an average spend of 1,11,275.32 INR                                                        │
│  4. Uber with an average spend of 1,07,809.51 INR                                                               │
│  5. IRCTC with an average spend of 1,05,685.65 INR                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 47039ff1-1771-4e9f-b10b-80615c3a2b75                                                                     │
│  Agent: Text2SQL Banking Analyst                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 28ca515b-71a8-410a-8f09-464da8d5ab8d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The top 5 average customer debit spend by merchant in 2025 are:                                  │
│                                                                                                                 │
│  1. Reliance Retail with an average spend of 1,17,148.32 INR                                                    │
│  2. Amazon India with an average spend of 1,14,694.58 INR                                                       │
│  3. Apple Store with an average spend of 1,11,275.32 INR                                                        │
│  4. Uber with an average spend of 1,07,809.51 INR                                                               │
│  5. IRCTC with an average spend of 1,05,685.65 INR                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✓ Completed (SQL: Captured)

All test queries processed!


In [71]:
# Create submission DataFrame
submission_df = pd.DataFrame(submission_results)

# Verify format
print(f"Submission DataFrame shape: {submission_df.shape}")
print(f"Columns: {list(submission_df.columns)}")
print(f"\nFirst few rows:")
print(submission_df.head())

# Verify no SQL code in responses
print("\nVerifying response format...")
for idx, row in submission_df.iterrows():
    response = str(row['response'])
    if "SELECT" in response.upper() and "FROM" in response.upper():
        # Check if it looks like SQL (but not just mentioning it)
        if not any(word in response for word in ["query", "SELECT statement", "SQL query"]):
            print(f"Warning: Row {idx + 1} might contain SQL code")

print("\nFormat verification complete!")

Submission DataFrame shape: (11, 2)
Columns: ['query', 'response']

First few rows:
                                               query  \
0       Top 5 merchants by debit spend in July 2025.   
1             Count active accounts by account type.   
2  List the top 3 spending categories by total tr...   
3  Top 3 customers who spent the most on dining i...   
4  Current balance for the top 5 accounts (all ti...   

                                            response  
0  The top 5 merchants by debit spend in July 202...  
1  The count of active accounts by account type i...  
2  The top 3 spending categories by total transac...  
3  The top 3 customers who spent the most on dini...  
4  The current balances for the top 5 accounts ar...  

Verifying response format...

Format verification complete!


In [72]:
# Import csv module for quoting constants
import csv

# Create all DataFrames
submission_df = pd.DataFrame(submission_results)
query_sql_df = pd.DataFrame(query_sql_results)
query_sql_insight_df = pd.DataFrame(query_sql_insight_results)

# Save submission.csv (original format: query, response)
# Use QUOTE_ALL to properly quote all fields (especially multi-line content)
submission_df.to_csv("submission.csv", index=False, quoting=csv.QUOTE_ALL, escapechar='\\')
print("✓ submission.csv saved successfully!")
print(f"  - {len(submission_df)} rows, {len(submission_df.columns)} columns: {', '.join(submission_df.columns)}")

# Save query_sql.csv (query, sql_query)
query_sql_df.to_csv("query_sql.csv", index=False, quoting=csv.QUOTE_ALL, escapechar='\\')
print("✓ query_sql.csv saved successfully!")
print(f"  - {len(query_sql_df)} rows, {len(query_sql_df.columns)} columns: {', '.join(query_sql_df.columns)}")

# Save query_sql_insight.csv (query, sql_query, insight)
# QUOTE_ALL ensures multi-line content in 'insight' column is properly quoted
query_sql_insight_df.to_csv("query_sql_insight.csv", index=False, quoting=csv.QUOTE_ALL, escapechar='\\')
print("✓ query_sql_insight.csv saved successfully!")
print(f"  - {len(query_sql_insight_df)} rows, {len(query_sql_insight_df.columns)} columns: {', '.join(query_sql_insight_df.columns)}")

# Display summary
print("\n" + "="*80)
print("FILES GENERATED")
print("="*80)
print("\n1. submission.csv (query, response):")
print(submission_df.head(3).to_string(index=False))
print("\n2. query_sql.csv (query, sql_query):")
print(query_sql_df.head(3).to_string(index=False))
print("\n3. query_sql_insight.csv (query, sql_query, insight):")
print(query_sql_insight_df.head(3).to_string(index=False))

✓ submission.csv saved successfully!
  - 11 rows, 2 columns: query, response
✓ query_sql.csv saved successfully!
  - 11 rows, 2 columns: query, sql_query
✓ query_sql_insight.csv saved successfully!
  - 11 rows, 3 columns: query, sql_query, insight

FILES GENERATED

1. submission.csv (query, response):
                                                         query                                                                                                                                                                                                                                                                                                             response
                  Top 5 merchants by debit spend in July 2025. The top 5 merchants by debit spend in July 2025 are:\n1. IRCTC with a total spend of 10,02,190.00 INR\n2. Zomato with a total spend of 9,65,256.00 INR\n3. Swiggy with a total spend of 9,65,091.00 INR\n4. Apple Store with a total spend of 6,88,285.00 INR\n5. Amaz